# WGS PiPeline

In [52]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np

sys.path.insert(0, '..')

from src.CCLE_postp_function import *
from JKBio import Datanalytics as da 
from JKBio import TerraFunction as terra
from JKBio import Helper as h
from JKBio.helper.google_sheet import GSheet
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display


%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

you need to have installed JKBio in the same folder as ccle_processing
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Loading BokehJS ...

## boot up

we are instanciating all the parameters needed for this pipeline to run

In [101]:
samplesetname = "20Q4"
prevname="20Q3"
prevversion = 0
prevprevname ='20Q2'
prevprevversion= 0
virtual_public='public-20q3-3d35'
virtual_dmc='dmc-20q3-033d'
virtual_internal='internal-20q3-00d0'

workspace1="terra-broad-cancer-prod/DepMap_WGS"
workspace2="terra-broad-cancer-prod/Getz_IBM_CellLines_WGS"


refworkspace="broad-firecloud-ccle/DepMap_WGS_CN"
cgaworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline-wgs"

source1="ccle"
source2="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
potential_list_url = "https://docs.google.com/spreadsheets/d/1BEgH03V4OmGhYeciLCZV00h6hp3WkO0basahS93akCE"
release = samplesetname

In [119]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)

refwm = dm.WorkspaceManager(refworkspace).disable_hound()
cgawm = dm.WorkspaceManager(cgaworkspace).disable_hound()

In [5]:
extract_to_change = {'from_arxspan_id': 'participant'}

In [273]:
potential_list = sheets.get(potential_list_url).sheets[0].to_frame().values.T[0].tolist()

In [338]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame().set_index('cds_sample_id')

you need to have installed JKBio in the same folder as ccle_processing


## Adding new data

We are looking for new samples in a range of workspaces.

They are quite messy and might contains duplicates, contain broken file paths...

- We are thus looking at the bam files one by one and comparing them with our own bams. 
- We remove broken files, duplicates and add new version of a cell line's bam if we find some.

In [56]:
# we will be missing "primary disease","sm_id", "cellosaurus_id", "gender, "age", "primary_site", "primary_disease", "subtype", "subsubtype", "origin", "comments"
#when SMid: match== 
samples, pairs, noarxspan = GetNewCellLinesFromWorkspaces(refworkspace, stype='wgs', refurl=refsheet_url, wmfroms = [workspace1, workspace2], sources=[source1, source2], match=['ACH-','CDS-'], participantslicepos=10, accept_unknowntypes=True, extract=extract_to_change, recomputedate=True)

refsamples is overrided by a refurl
Getting sample infos...

The shape of the sample tsv from <dalmatian.wmanager.WorkspaceManager terra-broad-cancer-prod/DepMap_WGS>: (25, 223)
Identifying any true duplicates by checking file hashes (this runs for each data source)...
This step can take a while as we need to use gsutil to check the size of each potential duplicate...
listing files in gs
These 0 bam file path do not exist: set()
listing files in gs


/home/jeremie/ccle_processing/src/CCLE_postp_function.py:231: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if extract['size'] not in samples.columns or recomputesize:


listing files in gs


/home/jeremie/ccle_processing/src/CCLE_postp_function.py:233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for k, val in samples.iterrows():


/home/jeremie/ccle_processing/src/CCLE_postp_function.py:242: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples[extract['release_date']] = list(h.datetoint(samples[extract['release_date']].values))
/home/jeremie/ccle_processing/src/CCLE_postp_function.py:243: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jeremie/ccle_processing/src/CCLE_postp_function.py:246: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

listing files in gs
listing files in gs
we found and removed 0 samples which did not match our id names: ['ACH-', 'CDS-']
found 0 likely replicate
listing files in gs
Len of samples before removal: 21
Dups from this workspace has len 0:
 []
Len of samples after removal: 21

The shape of the sample tsv from <dalmatian.wmanager.WorkspaceManager terra-broad-cancer-prod/Getz_IBM_CellLines_WGS>: (4, 233)
Identifying any true duplicates by checking file hashes (this runs for each data source)...
This step can take a while as we need to use gsutil to check the size of each potential duplicate...
listing files in gs
These 0 bam file path do not exist: set()
listing files in gs
listing files in gs
listing files in gs
listing files in gs
we found and removed 0 samples which did not match our id names: ['ACH-', 'CDS-']
found 0 likely replicate
listing files in gs
Len of samples before removal: 3
Dups from this workspace has len 0:
 []
Len of samples after removal: 3
we had 0 duplicates in the rel

In [57]:
noarxspan

,internal_bam_filepath,internal_bai_filepath,stripped_cell_line_name,arxspan_id,sequencing_date,participant_id,crc32c_hash,size
sample_id,,,,,,,,
CDS-ymRIxH,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,HCC-1588,HCC-1588,737552,PT-7zhvxCOS,2ldWlQ==,108374791873
CDS-5vtea4,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,MERO83_1,MERO83,737547,PT-Vd7k8elh,nRYbAw==,116306111418
CDS-X7YJtK,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,MERO95_1,MERO95,737547,PT-FJCfUk9B,YkQgKw==,117193717024
CDS-LvlS3j,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,TASK1_1,TASK1,737547,PT-ZxcFHknh,Xi90TA==,118399266655
CDS-R22IHj,gs://fc-49b66034-d2f1-4d65-84db-891ae1be11d4/G...,gs://fc-49b66034-d2f1-4d65-84db-891ae1be11d4/G...,MONO-MAC-1,MONO-MAC-1,734878,PT-G7iG07aJ,y2QITw==,152230340144


In [58]:
noarxspan.to_csv('temp/noarxspan_wgs_'+release+'.csv')

### finding back arxspan

In [59]:
noarxspan['arxspan_id'] = ['ACH-001078', 'ACH-001560', 'ACH-001562', 'ACH-001670', 'ACH-001129']

In [65]:
noarxspan = resolveFromWorkspace(noarxspan, refsamples = ccle_refsamples[ccle_refsamples['datatype'] == 'wgs'], match = ['ACH','CDS'], participantslicepos = 10, accept_unknowntypes = True, extract = extract_to_change)

we found and removed 0 samples which did not match our id names: ['ACH', 'CDS']
found 1 likely replicate
listing files in gs
Len of samples before removal: 5
Dups from this workspace has len 0:
 []
Len of samples after removal: 5


In [67]:
noarxspan

,internal_bam_filepath,internal_bai_filepath,stripped_cell_line_name,arxspan_id,sequencing_date,participant_id,crc32c_hash,size
sample_id,,,,,,,,
CDS-ymRIxH,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,HCC-1588,ACH-001078,737552,PT-7zhvxCOS,2ldWlQ==,108374791873
CDS-5vtea4,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,MERO83_1,ACH-001560,737547,PT-Vd7k8elh,nRYbAw==,116306111418
CDS-X7YJtK,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,MERO95_1,ACH-001562,737547,PT-FJCfUk9B,YkQgKw==,117193717024
CDS-LvlS3j,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,TASK1_1,ACH-001670,737547,PT-ZxcFHknh,Xi90TA==,118399266655
CDS-R22IHj,gs://fc-49b66034-d2f1-4d65-84db-891ae1be11d4/G...,gs://fc-49b66034-d2f1-4d65-84db-891ae1be11d4/G...,MONO-MAC-1,ACH-001129,734878,PT-G7iG07aJ,y2QITw==,152230340144


In [63]:
samples = pd.concat([samples, noarxspan], sort=False)

In [70]:
samples = assessAllSamples(samples, ccle_refsamples, stype='wgs', rename={}, extract={})

you need to have installed JKBio in the same folder as ccle_processing
we had 0 duplicates in the release buckets


In [71]:
samples

,internal_bam_filepath,internal_bai_filepath,stripped_cell_line_name,arxspan_id,sequencing_date,participant_id,crc32c_hash,size,datatype,version
sample_id,,,,,,,,,,
CDS-ttmLuC,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,C396,ACH-001818,737547,PT-oZH9BTUX,wP5Uig==,118951856040,wgs,1
CDS-VVWDLi,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,CHP134,ACH-001338,737547,PT-sqRdd0QF,TKjC7Q==,111261029620,wgs,1
CDS-cjMTSm,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,HCC1739BL,ACH-001828,737552,PT-OCJbBZ3S,qID1QA==,113315538347,wgs,1
CDS-AQpAIt,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,KARPAS1718,ACH-001533,737552,PT-ceOWs8HZ,vaScFg==,114662826746,wgs,1
CDS-cXQOkP,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,KOSC2CL343,ACH-001543,737547,PT-gCMWGzPt,gRp/bw==,96804199449,wgs,1
CDS-YLQJ19,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,M140325,ACH-002511,737552,PT-ELEMMjP8,87LwZQ==,125056253166,wgs,1
CDS-AqZLna,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,M1610113,ACH-002512,737547,PT-pUBtKkYR,7TbnGw==,110877810899,wgs,1
CDS-3PGQ84,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,MEGA2,ACH-001553,737560,PT-SMrpbPt7,GGyO/Q==,100818382371,wgs,1
CDS-nKfczI,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,MEL270,ACH-002014,737547,PT-RkMpD0qb,OHynTg==,109896913581,wgs,1


## getting the addtional data and writing it here in the right order 'as shown above'
- use the stripped_cell_line_name to find the samples on https://docs.google.com/spreadsheets/d/1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE/edit#gid=356471436. 
- Make sure that we don't have duplicate cell lines in there. Otherwise, use the duplicate renaming function
- copy Primary Site, Primary Disease, Subtype, Comments, Disease Sub-subtype, if they exist. (sometimes subtype and subsubtype are the same.. don't use subsubtype then.
- look for the cell line in cellosaurus, you might need to use one of the aliases given in master depmap pv..
- copy  cellosaurus_id gender age info or write 'U' if they don't exist. 'can be a number or {Embryonic, Children, Adult, Fetus, U} 
- check that it does not say this cell line is not a duplicate from another cell line
- check that if it says this cell line is derived/children/father/samepatient from other cell lines, and that if we have any of the other cell lines, that the patient id is changed to be the same one for all (be sure that you are updating everywhere these patient ids are used)

In [75]:
# If I have a previous samples I can update unknown data directly
index=[]
notfound=[]
toupdate = {"sex":[],
"primary_disease":[],
"cellosaurus_id":[],
"age":[],
"primary_site":[],
"subtype":[],
"subsubtype":[],
"comments":[],
"stripped_cell_line_name":[],
"parent_cell_line":[''],
"matched_normal":[''],
"participant_id":[]}
for k, val in samples.iterrows():
    dat = ccle_refsamples[ccle_refsamples['arxspan_id']==val['arxspan_id']]
    if len(dat)>0:
        index.append(k)
        for k, v in toupdate.items():
            toupdate[k].append(dat[k].tolist()[0])
    else:
        notfound.append(k)
# doing so..
for k, v in toupdate.items():
    samples.loc[index,k] =v
len(samples.loc[notfound].patient_id),samples.loc[notfound].patient_id.tolist()

In [78]:
# for these samples I will need to check and manually add the data in the list 
samples.loc[notfound]

,internal_bam_filepath,internal_bai_filepath,stripped_cell_line_name,arxspan_id,sequencing_date,participant_id,crc32c_hash,size,datatype,version,sex,primary_disease,cellosaurus_id,age,primary_site,subtype,subsubtype,comments
sample_id,,,,,,,,,,,,,,,,,,
CDS-cjMTSm,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,HCC1739BL,ACH-001828,737552,PT-OCJbBZ3S,qID1QA==,113315538347,wgs,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CDS-3PGQ84,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,MEGA2,ACH-001553,737560,PT-SMrpbPt7,GGyO/Q==,100818382371,wgs,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CDS-deAUlH,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,MM28__,ACH-002013,737547,PT-ws3gMJ8I,BVyqdQ==,118332580889,wgs,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CDS-vp6X0E,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,VP267,ACH-001705,737552,PT-gkuA6nS5,XIf9Ew==,114853944490,wgs,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CDS-bTfRlF,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,gs://fc-d2a92869-a028-40cf-a400-6ce6f7a5e92b/D...,WAOSEL,ACH-001707,737552,PT-heEmqvCI,AMuCew==,90677570187,wgs,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CDS-OQITe6,gs://fc-49b66034-d2f1-4d65-84db-891ae1be11d4/G...,gs://fc-49b66034-d2f1-4d65-84db-891ae1be11d4/G...,CORL26,ACH-001046,737546,PT-Z2Fq3yte,7+vYIA==,100035318330,wgs,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
samples = changeCellLineNameInNewSet(new = samples, ref=ccle_refsamples, datatype="wgs", dupdict=dup)

In [88]:
toupdate = {"sex":["Female","Male","Male","Female","Male","Female"],
"primary_disease":["Breast Cancer","Leukemia","Eye Cancer","Breast Cancer","Leukemia","Lung Cancer"],
"cellosaurus_id":["CVCL_3280","CVCL_1832","CVCL_4D15","CVCL_2755","CVCL_Y549","CVCL_2410"],
"age":[51,24,72,48,"Adult","U"],
"primary_site":["pleural_effusion","haematopoietic_and_lymphoid_tissue","liver","breast","haematopoietic_and_lymphoid_tissue","bone_marrow"],
"subtype":["Carcinoma","CML","Uveal Melanoma","Carcinoma","CLL","Small Cell Lung Cancer (SCLC)"],
"subsubtype":["breast_ductal_carcinoma","","","breast_ductal_carcinoma","b_cell",""],
"comments":["Transformant: NCBI_TaxID; 10376; Epstein-Barr virus (EBV). TNBC", "","","Derived from a breast cancer biopsy removed from a 48 year-old female. The histology of the breast cancer biopsy was defined as Ductal Grade III, tumour cells did not stain positive for oestrogen receptors. The VP267 cell line was derived from the same in","",""],
"parent_cell_line":['','','','','',''],
"matched_normal":['','','','','',''],
"participant_id":['PT-OCJbBZ3S','PT-SMrpbPt7',"PT-ws3gMJ8I","PT-n3CSJEGT",'PT-y3RbI7uD',"PT-Z2Fq3yte"]}

In [89]:
a  = pd.DataFrame(toupdate)
a['name'] = samples.loc[notfound,"stripped_cell_line_name"].tolist()
a

,sex,primary_disease,cellosaurus_id,age,primary_site,subtype,subsubtype,comments,participant_id,name
0,Female,Breast Cancer,CVCL_3280,51,pleural_effusion,Carcinoma,breast_ductal_carcinoma,Transformant: NCBI_TaxID; 10376; Epstein-Barr ...,PT-OCJbBZ3S,HCC1739BL
1,Male,Leukemia,CVCL_1832,24,haematopoietic_and_lymphoid_tissue,CML,,,PT-SMrpbPt7,MEGA2
2,Male,Eye Cancer,CVCL_4D15,72,liver,Uveal Melanoma,,,PT-ws3gMJ8I,MM28__
3,Female,Breast Cancer,CVCL_2755,48,breast,Carcinoma,breast_ductal_carcinoma,Derived from a breast cancer biopsy removed fr...,PT-n3CSJEGT,VP267
4,Male,Leukemia,CVCL_Y549,Adult,haematopoietic_and_lymphoid_tissue,CLL,b_cell,,PT-y3RbI7uD,WAOSEL
5,Female,Lung Cancer,CVCL_2410,U,bone_marrow,Small Cell Lung Cancer (SCLC),,,PT-Z2Fq3yte,CORL26


In [90]:
# updating..
for k, v in toupdate.items():
    samples.loc[notfound,k] =v

In [84]:
# uploading to our bucket (now a new function)
h.changeToBucket(samples,'gs://cclebams/wgs/', name_col= "index" , values=['internal_bam_filepath','internal_bai_filepath'], filetypes=['bam', 'bai'], catchdup=True, test=False)

,internal_bam_filepath,internal_bai_filepath,stripped_cell_line_name,arxspan_id,sequencing_date,participant_id,crc32c_hash,size,datatype,version,sex,primary_disease,cellosaurus_id,age,primary_site,subtype,subsubtype,comments,gender
sample_id,,,,,,,,,,,,,,,,,,,
CDS-ttmLuC,gs://cclebams/wgs/CDS-ttmLuC.bam,gs://cclebams/wgs/CDS-ttmLuC.bai,C396,ACH-001818,737547,PT-oZH9BTUX,wP5Uig==,118951856040,wgs,1,U,Bone Cancer,U,U,bone,osteosarcoma,NaN,"Received from Academic lab (Gorlick, Montefior...",NaN
CDS-VVWDLi,gs://cclebams/wgs/CDS-VVWDLi.bam,gs://cclebams/wgs/CDS-VVWDLi.bai,CHP134,ACH-001338,737547,PT-sqRdd0QF,TKjC7Q==,111261029620,wgs,1,Male,Neuroblastoma,CVCL_1124,43952,autonomic_ganglia,neuroblastoma,NaN,MYCN amp; adrenergic ; 1p - (literature),NaN
CDS-cjMTSm,gs://cclebams/wgs/CDS-cjMTSm.bam,gs://cclebams/wgs/CDS-cjMTSm.bai,HCC1739BL,ACH-001828,737552,PT-OCJbBZ3S,qID1QA==,113315538347,wgs,1,NaN,Breast Cancer,CVCL_3280,51,pleural_effusion,Carcinoma,breast_ductal_carcinoma,Transformant: NCBI_TaxID; 10376; Epstein-Barr ...,Female
CDS-AQpAIt,gs://cclebams/wgs/CDS-AQpAIt.bam,gs://cclebams/wgs/CDS-AQpAIt.bai,KARPAS1718,ACH-001533,737552,PT-ceOWs8HZ,vaScFg==,114662826746,wgs,1,Male,Lymphoma,U,U,haematopoietic_and_lymphoid_tissue,non_hodgkin_lymphoma,splenic_lymphoma,NaN,NaN
CDS-cXQOkP,gs://cclebams/wgs/CDS-cXQOkP.bam,gs://cclebams/wgs/CDS-cXQOkP.bai,KOSC2CL343,ACH-001543,737547,PT-gCMWGzPt,gRp/bw==,96804199449,wgs,1,Male,Head and Neck Cancer,U,51.0,upper_aerodigestive_tract,upper_aerodigestive_squamous,oral,Human Tumor cells derived from oral cancer at ...,NaN
CDS-YLQJ19,gs://cclebams/wgs/CDS-YLQJ19.bam,gs://cclebams/wgs/CDS-YLQJ19.bai,M140325,ACH-002511,737552,PT-ELEMMjP8,87LwZQ==,125056253166,wgs,1,Female,Skin Cancer,U,64.0,skin,melanoma,mucosal,Biopsy site - vagina; Received from Academic l...,NaN
CDS-AqZLna,gs://cclebams/wgs/CDS-AqZLna.bam,gs://cclebams/wgs/CDS-AqZLna.bai,M1610113,ACH-002512,737547,PT-pUBtKkYR,7TbnGw==,110877810899,wgs,1,Male,Skin Cancer,NaN,U,skin,Melanoma,acral,M1610113 (p9); Mutation: NRASG12R; biopsy site...,NaN
CDS-3PGQ84,gs://cclebams/wgs/CDS-3PGQ84.bam,gs://cclebams/wgs/CDS-3PGQ84.bai,MEGA2,ACH-001553,737560,PT-SMrpbPt7,GGyO/Q==,100818382371,wgs,1,NaN,Leukemia,CVCL_1832,24,haematopoietic_and_lymphoid_tissue,CML,,,Male
CDS-nKfczI,gs://cclebams/wgs/CDS-nKfczI.bam,gs://cclebams/wgs/CDS-nKfczI.bai,MEL270,ACH-002014,737547,PT-RkMpD0qb,OHynTg==,109896913581,wgs,1,Male,Eye Cancer,U,62.0,uvea,uveal_melanoma,NaN,"Mel270, OMM23, and OMM25 are from the same pat...",NaN


In [ ]:
names=[]
subccle_refsamples = ccle_refsamples[ccle_refsamples['datatype'] == "wgs"]
for k, val in samples.iterrows():
    val = val["arxspan_id"]
    names.append(val)
    samples.loc[k, 'version'] = len(subccle_refsamples[subccle_refsamples['arxspan_id'] == val]) + names.count(val)
samples['version'] = samples['version'].astype(int)

In [110]:
ccle_refsamples = pd.read_csv('temp/updated_ref_samples.csv',index_col=0)

In [ ]:
subccle_refsamples.sequencing_date = h.datetoint(subccle_refsamples.sequencing_date.values, split='/', order = "asc")
for k, val in samples.iterrows():
    loc = subccle_refsamples[subccle_refsamples.arxspan_id==val.arxspan_id]
    if len(loc)>0:
        if val.sequencing_date > 0:
            for i, v in loc.iterrows():
                if v.sequencing_date > val.sequencing_date:
                    ccle_refsamples.loc[i,'version']+=1
                    samples.loc[k, 'version']-=1
        else:
            if max(loc['size']) > val['size']:
                samples.loc[k, 'version'] = 1
                ccle_refsamples.loc[loc.index,'version'] = ccle_refsamples.loc[loc.index,'version'].values+1     

In [112]:
ccle_refsamples = ccle_refsamples.append(samples, sort=False)

In [215]:
for val in samepatient:
    sub = ccle_refsamples[ccle_refsamples.arxspan_id.isin(val)]
    if len(set(sub.participant_id))>2:
        print('we found a missig participant relationship')
        # ccle_refsamples.loc[ccle_refsamples.index, "participant_id"]=sub.participant_id[0]

In [ ]:
ccle_refsamples.to_csv('temp/updated_ref_samples.csv')

In [116]:
#pairs = setupPairsFromSamples(samples, subccle_refsamples, extract={'patient_id':'participant_id'})

NameError: name 'subccle_refsamples' is not defined

In [118]:
#uploading new samples to mut
refwm.upload_samples(samples)
#refwm.upload_entities('pairs', pairs)
#refwm.update_pair_set(pair_set_id=samplesetname,pair_ids=pairs.index)
sam = refwm.get_samples()

#pair = refwm.get_pairs()
#refwm.update_pair_set(pair_set_id='all',pair_ids=pair.index)
#creating a sample set
refwm.update_sample_set(sample_set_id=samplesetname, sample_ids=samples.index)
refwm.update_sample_set(sample_set_id='all', sample_ids=[i for i in sam.index.tolist() if i!='nan'])

Successfully imported 29 participants.
Successfully imported 29 samples.
Successfully imported 1 sample sets:
  * 20Q4 (29 samples)
Successfully imported 1 sample sets:
  * all (359 samples)


In [138]:
#and CGA
cgawm.upload_samples(samples)
sam = cgawm.get_samples()
#creating a sample set
cgawm.update_sample_set(sample_set_id=samplesetname, sample_ids=samples.index)
cgawm.update_sample_set(sample_set_id='all', sample_ids=[i for i in sam.index.tolist() if i!='nan'])

Successfully imported 1 sample sets:
  * all (359 samples)


## Check that we have all the cell lines we expect for this release

This involves comparing to the list in the Google sheet "Cell Line Profiling Status."

_As the list cannot be parsed, we are not comparing it for now_

In [ ]:
# this function may not work - it hasn't been tested
url = 'https://docs.google.com/spreadsheets/d/1qus-9TKzqzwUMNWp8S1QP4s4-3SsMo2vuQRZrNXf7ag/edit?ts=5db85e27#gid=0&fvid=1627883727'

compareToCuratedGS(url, sample = newsample[0], samplesetname = samplesetname, colname = 'CN New to internal')

# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 5 functions To generate the mutation dataset:

*   For new samples in DepMap, run the ICE version of this task. CCLE2 samples used Agilent targets, so this pipeline should be used instead. The pipelines are identical in terms of their outputs, but the proper targets, baits, and pseudo normal should be used based on how the samples were sequenced.

    **ICE_CGA_Production_Analysis_Pipeline_Cell_Lines_copy** (cclf/CGA_Production_Analysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22) OR


    **AGILENT_CGA_Production_Analysis_Pipeline_Cell_Lines** (cclf/CGA_Production_Anablysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22)

*   **common_variant_filter** (breardon/common_variant_filterSnapshot ID: 3)
*   **filterMAF_on_CGA_pipeline** (gkugener/filterMAF_on_CGA_pipelineSnapshot ID: 8)
*   **aggregateMAFs_selectFields** (ccle_mg/aggregateMAFs_selectFieldsSnapshot ID: 1)

This outputs to be downloaded will be saved in the sample set that was run. The output we use for the release is:


*   **passedCGA_filteredMAF_aggregated** 

There are several other tasks in this workspace. In brief:



*   **CGA_Production_Analysis_Pipeline_Cell_Lines** (lelagina/CGA_Production_Analysis_Pipeline_Cell_LinesSnapshot ID: 12). This task is the same as the ICE and AGILENT prefixed version above, except that it relied on pulling the baits and targets to use from the metadata stored for the samples. Having AGILENT and ICE versions specified made the uploading and running process easier.
*   **SANGER_CGA_Production_Analysis_Pipeline_Cell_Lines** (cclf/CGA_Production_Analysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22). This task was trying to run the CGA pipeline on the Sanger WES data, using a Sanger pseudo normal. In its current implementation, this task fails to complete for the samples.
*   **UNFILTERED_aggregateMAFs_selectFields** (ccle_mg/aggregateMAFs_selectFieldsSnapshot ID: 1). Aggregates the MAF outputted by the CGA cell line pipeline prior to the common variant filter and germline filtering tasks. This can give us insight to which mutations are getting filtered out when. We may want to potentially include this MAF in the release so people can see why certain mutations of interest may be getting filtered out.
*   WES_DM_Mutation_Calling_Pipeline_(standard |expensive) (gkugener/WES_DM_Mutation_Calling_PipelineSnapshot ID: 2). This was a previous mutation calling pipeline implemented for CCLE. We do not use this pipeline any more as the CGA pipeline looks better.
*   aggregate_filterMAF_CGA (CCLE/aggregate_filterMAF_CGASnapshot ID: 1). An aggregation MAF task that we used in the past. We do not use this task anymore.
*   calculate_mutational_burden (breardon/calculate_mutational_burdenSnapshot ID: 21). This task can be used to calculate the mutational rate of the samples. We do not make use of this data in the release although it could be of interest.
*   summarizeWigFile (breardon/summarizeWigFileSnapshot ID: 5). CCLF ran this task (might be necessary for the mutational burden task). For our workflow, we do not run it.

## On Terra

### realigning to hg19

In [249]:
unmap = "BamToUnmappedRGBams"
groupfiles= "ArrayOfFilesToTxt"
mapping = "PreProcessingForVariantDiscovery_GATK4"

In [151]:
submission_id2 = refwm.create_submission(unmap, samplesetname,'sample_set',expression='this.samples')

Successfully created submission 5e61d3db-3cdd-4f20-9608-d67fb06a4905.


In [245]:
a = cgawm.get_samples().loc[cgawm.get_sample_sets().loc[samplesetname].samples]

In [251]:
for k, i in a.iterrows():
    refwm.update_sample_attributes({'ubam_file_list': i['file_list_name']}, sample_id=k)

Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_list']' for 1 samples.
Successfully updated attributes '['ubam_file_lis

### calling mutations

In [161]:
submission_id1 = cgawm.create_submission("CGA_WES_CCLE_Characterization_Pipeline_v0.1_Jul2019_copy", samplesetname,'sample_set',expression='this.samples')

Successfully created submission fb0ba43c-fb90-4c92-a0e1-f92d9c35c51e.


In [238]:
terra.waitForSubmission(cgaworkspace, submission_id2)
submission_id2 = cgawm.create_submission(groupfiles, samplesetname,'sample_set',expression='this.samples')

Successfully created submission c9d45fdd-9578-417f-a1bf-b406622e542c.


In [252]:
terra.waitForSubmission(refworkspace, submission_id2)
submission_id2 = refwm.create_submission(mapping, samplesetname,'sample_set',expression='this.samples')

Successfully created submission c79fd133-58ac-4ac5-9742-414bf3f9e48e.


In [255]:
terra.waitForSubmission(refworkspace, submission_id2)
submission_id2 = refwm.create_submission("cnn-variant-filter", samplesetname,'sample_set',expression='this.samples')
submission_id3 = refwm.create_submission("CNV_Somatic_Workflow_on_Sample", samplesetname,'sample_set',expression='this.samples')
submission_id4 = refwm.create_submission("Manta_SomaticSV", samplesetname,'sample_set',expression='this.samples')

Successfully created submission 37ea7504-48da-4169-b9a9-ca65ae53abad.
Successfully created submission 55dd3446-214f-4421-87d9-e385c53e99fb.
Successfully created submission 601a257a-2704-4180-93c5-4cb706d64154.


In [259]:
terra.waitForSubmission(cgaworkspace, submission_id1)
submission_id1 = cgawm.create_submission("common_variant_filter", samplesetname, 'sample_set', expression='this.samples')

Successfully created submission db715982-826a-45ee-a2f8-43705ba746a4.


In [260]:
terra.waitForSubmission(cgaworkspace, submission_id1)
submission_id1 = cgawm.create_submission("filterMAF_on_CGA_pipeline", samplesetname,'sample_set',expression='this.samples')

1.0 of jobs Succeeded in submission 0.sion 0. 8 mn elapsed..
Successfully created submission 0257e1e2-f84d-4478-942a-62c5790da1bc.


### aggregations

In [264]:
#filtered
terra.waitForSubmission(cgaworkspace, submission_id1)
submission_id1 = cgawm.create_submission("aggregateMAFs_selectFields_copy", samplesetname)
#unfiltered 
submission_id5 = cgawm.create_submission("aggregateMAFs_selectFields_unfiltered", samplesetname)

Successfully created submission e7387274-2f53-4ab3-943c-967107094a60.


In [265]:
terra.waitForSubmission(refworkspace, [submission_id2,submission_id3,submission_id4])
submission_id2 = refwm.create_submission("aggregate_vcfs_sv", "all")
submission_id3 = refwm.create_submission("aggregate_vcfs", "all")
submission_id4 = refwm.create_submission("aggregate_vcfs_snp", "all")
submission_id6 = refwm.create_submission("aggregate_vcfs_sv_somatic", "all")
submission_id7 = refwm.create_submission("Aggregate_CN_seg_files", "all")

Successfully created submission 58e2c1d1-47a6-4540-8e53-1a2fcfa2dc7e.
Successfully created submission 21c66ea6-14e0-4fb9-a1b5-924b4c566b4d.


In [ ]:
terra.waitForSubmission(refworkspace, [submission_id2, submission_id3, submission_id4, submission_id6, submission_id7])
terra.waitForSubmission(cgaworkspace, [submission_id1, submission_id5])

### copy pairs data to sample data

In [ ]:
pairs = refwm.get_pairs()

In [ ]:
pairs = pairs[pairs.index.isin(tokeep)]
pairs = pairs[~pairs['mutation_validator_validated_maf'].isna()]
pairs = pairs.drop(columns=['case_sample','control_sample','participant_id'])
pairs.index = [i.split('_')[0] for i in pairs.index]

In [ ]:
refwm.update_sample_attributes(pairs)

### Save the workflow configurations used

In [ ]:
terra.saveConfigs(refworkspace,'./data/'+samplesetname+'/Mutconfig')

## On local


### Remove some datafile to save money¶

In [ ]:
val = ""
! gsutil -m rm gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/$val/**/call-tumorMM_Task/*.cleaned.bam

In [301]:
# sometimes it does not work so better check again
a = refwm.get_samples()
e = []
for i in a["readgroup_ubams"].values.tolist():
    if i is not np.nan:
        e.extend(i)
gcp.rmFiles(e)
toremove = ["readgroup_ubams",]
for val in toremove:
    refwm.disable_hound().delete_entity_attributes('sample', res[val])

KeyError: 'readgroup_ubams'

In [309]:
ccle_refsamples = pd.read_csv('temp/updated_ref_samples.csv', index_col=0)

In [315]:
#move HG38 to our bucket
onlycol = ['analysis_ready_bam', 'analysis_ready_bam_index', 'analysis_ready_bam_md5']
samplesinset= [i['entityName'] for i in refwm.get_entities('sample_set').loc[samplesetname].samples]
wgs_newgs = 'gs://cclebams/wgs_hg38/'
wgs_res, _ = terra.changeGSlocation(refworkspace, newgs=wgs_newgs, onlysamples=samplesinset, onlycol=onlycol, entity='sample', keeppath=False, dry_run = False)

using the data from broad-firecloud-ccle/DepMap_WGS_CN sample list
this should only contains gs:// paths otherwise precise columns using "onlycol"
Successfully updated attributes '['analysis_ready_bam', 'analysis_ready_bam_index', 'analysis_ready_bam_md5']' for 29 samples.


In [354]:
ccle_refsamples.loc[wgs_res.index.tolist()][['legacy_bam_filepath','legacy_bai_filepath','legacy_size',	'legacy_crc32c_hash']] = ccle_refsamples.loc[wgs_res.index.tolist()][['internal_bam_filepath','internal_bai_filepath','size','crc32c_hash']]
ccle_refsamples.loc[wgs_res.index.tolist()][['internal_bam_filepath','internal_bai_filepath']] = wgs_res[['analysis_ready_bam','analysis_ready_bam_index']]
ccle_refsamples.loc[wgs_res.index.tolist(),'size'] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(wgs_res['analysis_ready_bam'].tolist(),'-l')]
ccle_refsamples.loc[wgs_res.index.tolist(),'crc32c_hash'] = [gcp.extractHash(i) for i in gcp.lsFiles(wgs_res['analysis_ready_bam'].tolist(),'-L')]
ccle_refsamples.loc[wgs_res.index.tolist(),'md5_hash'] = gcp.catFiles(wgs_res['analysis_ready_bam_md5'].tolist(), cut=32)

listing files in gs
listing files in gs
0.0


### get QC files

In [359]:
dataMut = getQC(workspace=cgaworkspace ,only=[], qcname=["gatk_cnv_all_plots", "lego_plotter_pngs", "copy_number_qc_report", "ffpe_OBF_figures", "mut_legos_html", "oxoG_OBF_figures", "tumor_bam_base_distribution_by_cycle_metrics", "tumor_bam_converted_oxog_metrics"])

In [360]:
dataBamCGA = getQC(workspace=cgaworkspace ,only=[], qcname=["tumor_bam_alignment_summary_metrics", "tumor_bam_bait_bias_summary_metrics", "tumor_bam_gc_bias_summary_metrics", "tumor_bam_hybrid_selection_metrics", "tumor_bam_insert_size_histogram", "tumor_bam_insert_size_metrics", "tumor_bam_pre_adapter_summary_metrics", "tumor_bam_quality_by_cycle_metrics", "tumor_bam_quality_distribution_metrics", "tumor_bam_quality_yield_metrics"])

In [365]:
dataBamGATK = getQC(workspace=refworkspace, only=[], qcname=["duplication_metrics", "bqsr_report"])

In [367]:
dataCN = getQC(workspace=refworkspace,only=[], qcname=["allelic_counts_tumor","delta_MAD_tumor","denoised_MAD_tumor","scaled_delta_MAD_tumor","denoised_copy_ratios_lim_4_plot_tumor","denoised_copy_ratios_plot_tumor","modeled_segments_plot_tumor"])

In [371]:
for k,v in dataMut.items():
    if k =='nan':
        continue
    ccle_refsamples.loc[k,'processing_qc'] = str(v) + ',' #+ ccle_refsamples.loc[k,'processing_qc']
for k,v in dataCN.items():
    if k =='nan':
        continue
    ccle_refsamples.loc[k,'processing_qc'] = str(v) + ',' + ccle_refsamples.loc[k,'processing_qc']
for k,v in dataBamGATK.items():
    if k =='nan':
        continue
    ccle_refsamples.loc[k,'bam_qc'] = str(v) + ',' #+ ccle_refsamples.loc[k,'bam_qc']
for k,v in dataBamCGA.items():
    if k =='nan':
        continue
    ccle_refsamples.loc[k,'bam_qc'] = str(v) + ',' + ccle_refsamples.loc[k,'bam_qc']

### downloading from terra

In [356]:
res = refwm.get_sample_sets().loc["all"]
res

you need to have installed JKBio in the same folder as ccle_processing


samples              [CDS-00Nrci, CDS-0sdrSB, CDS-0Z9yIY, CDS-17moN...
merged_vcf           gs://fc-secure-bd7b8bc9-f665-4269-997e-5a40208...
combined_seg_file    gs://fc-secure-bd7b8bc9-f665-4269-997e-5a40208...
Name: all, dtype: object

In [373]:
snps = res['merged_vcf']
segments = res['combined_seg_file']
! gsutil cp $segments "temp/segments_terra_merged_wgs.seg"
! gsutil cp $snps gs://cclebams/germline_data/
! gsutil -m acl ch -ru taiga-892@cds-logging.iam.gserviceaccount.com:R gs://cclebams/germline_data/all.called.vcf

Copying gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/2493f472-63da-4d2d-83c0-a6217e06d3a4/aggregate_CN_segments_wrkflw/cd7cd515-1710-4818-98fa-02b8a5e1afd0/call-aggregate_CN_segments/all.called.seg...
\ [1 files][138.8 MiB/138.8 MiB]                                                
Operation completed over 1 objects/138.8 MiB.                                    


In [ ]:
res = cgawm.get_sample_sets().loc["all"]
res

In [ ]:
filtered = res['filtered_CGA_MAF_aggregated']
! gsutil cp $filtered "temp/mutation_filtered_terra_merged_wgs.txt"

In [ ]:
unfiltered = res['unfiltered_CGA_MAF_aggregated']
! gsutil cp $unfiltered "temp/mutation_unfiltered_terra_merged.txt"

In [374]:
segments = pd.read_csv('temp/segments_terra_merged_wgs.seg', sep='\t')
#new_refsamples = pd.read_csv('temp/newrefCN.csv',index_col="cds_sample_id")
#new_refsamples = pd.read_csv('temp/newrefCN.csv',index_col="cds_sample_id")
#new_refsamples = pd.read_csv('temp/newrefCN.csv',index_col="cds_sample_id")
#new_refsamples = pd.read_csv('temp/newrefCN.csv',index_col="cds_sample_id")

In [376]:
len(set(segments.Sample))

359

In [377]:
segments = segments.rename(columns={'CONTIG':'Chromosome',
'START':'Start',
'END':'End',
'Sample':"DepMap_ID",
'NUM_POINTS_COPY_RATIO':'Num_Probes',
'MEAN_LOG2_COPY_RATIO':'Segment_Mean',
'CALL':'Status'})
segments['Source'] = "Broad WGS"

In [380]:
renaming = removeOlderVersions(names = set(segments['DepMap_ID']), refsamples = refwm.get_samples(), arxspan_id = "arxspan_id", version="version")

removed 2 duplicate samples


NameError: name 'file' is not defined

In [383]:
segments = segments[segments['DepMap_ID'].isin(renaming.keys())].replace({'DepMap_ID':renaming})

#### saving samples used for 20Q2

In [385]:
ccle_refsamples.loc[renaming.keys(),samplesetname]=1
ccle_refsamples.to_csv('temp/updated_ref_samples.csv', index=None)

#### knowing the missing samples

In [336]:
wgs  = set(refwm.get_samples().arxspan_id)

In [337]:
%store -r notWESnotlegacy 
notWESnotlegacy - wgs

{'ACH-001714', 'ACH-002709', 'ACH-002874', 'ACH-002875'}

### postprocessing


Here, rather than rerunning the entire analysis, because we know we are adding only WES samples, we can download the previous release's MAF, add the samples, update any annotations, and perform any global filters at the end.

First we need to do an additional step of filtering on coverage and number 

- readMutations
- createSNPs
- addToMainMutation
- filterAllelicFraction
- filterMinCoverage
- mergeAnnotations
- addAnnotation
- maf_add_variant_annotations
- mutation_maf_to_binary_matrix (x3)

In [ ]:
gene_mapping = tc.get(name='depmap-wes-cn-data--08f3', file='WES_CN_hg38_gene_DM19Q1')
gene_mapping = gene_mapping[gene_mapping.columns[:5]]
gene_mapping = gene_mapping.rename(columns={'EGID':'ensembl_id','SYMBOL':'symbol','CHR':'Chromosome','CHRLOC':'start','CHRLOCEND':'end'})

In [ ]:
segments['Chromosome'] = [i[3:] for i in segments['Chromosome'].tolist()]
segments = segments.sort_values(by=['DepMap_ID','Chromosome','Start','End'])
gene_mapping = gene_mapping.sort_values(by=['Chromosome','start','end'])

In [ ]:
segments.to_csv('temp/WGS_segments_all_'+samplesetname+'.csv', index=None)

In [ ]:
gapmergedsegs = manageGapsInSegments(segments)

In [ ]:
genecn = toGeneMatrix(gapmergedsegs, gene_mapping)

In [ ]:
genecn = np.log2((2**genecn)+1)
genecn.mean(1)

In [ ]:
%%R
newly_merged_maf <- readMutations('temp/mutation_filtered_terra_merged.txt')
new_release <- createSNPs(newly_merged_maf)
names(new_release)

In [ ]:
%%R
previous.release.maf <- load.from.taiga(data.name='depmap-mutations-maf-35fe', data.file=paste0('mutations.',prevname),data.version=prevversion)
if (colnames(previous.release.maf)[1] == 'X1' || colnames(previous.release.maf)[1] == "") {
 previous.release.maf[,1] <- NULL 
}
prevnames <- names(previous.release.maf)
prevnames

In [ ]:
%%R
print(nrow(previous.release.maf))
merged <- addToMainMutation(previous.release.maf, new_release)
print(nrow(merged))

In [ ]:
%%R
## Adding more
newly_merged_maf <- readMutations('temp/mutation_filtered_terra_merged.txt')
new_release <- createSNPs(newly_merged_maf)
print(names(new_release))
merged <- addToMainMutation(merged, new_release)
nrow(merged)

In [ ]:
%%R
setdiff(names(merged), names(previous.release.maf))

In [ ]:
%%R
## check if some rows have nans
length(which(is.na(merged$Hugo_Symbol)))

In [ ]:
%%R
dim(merged)

In [ ]:
%%R
filtered <- filterAllelicFraction(merged)

In [ ]:
%%R
head(new_release[which(new_release$Tumor_Sample_Barcode=='ACH-002466'),])

In [ ]:
%%R
filtered <- filterMinCoverage(filtered$merged, filtered$removed_from_maf)

In [ ]:
%%R
head(merged)

In [ ]:
%%R
clean_annotations <- mergeAnnotations(merged,previous.release.maf)

In [ ]:
%%R
# Guillaume's version
new_release <- addAnnotation(filtered$merged, clean_annotations, colnames(previous.release.maf))
# Allie's version
new_release <- maf_add_variant_annotations(new_release)

In [ ]:
%%R
# some matric files that does get used internaly and might be useful
damaging_mutation <- mutation_maf_to_binary_matrix(new_release, damaging =  TRUE)
other_mutation <- mutation_maf_to_binary_matrix(new_release, other = TRUE)
hotspot_mutation <- mutation_maf_to_binary_matrix(new_release, hotspot = TRUE)

In [ ]:
%%R
# Save the ready to upload file to upload to taiga
write.table(
  new_release, 
  paste0('temp/mutations.', release, '.all.csv'), sep = ',', quote = F, row.names = F)
# Save the ready to upload file to upload to taiga
write.table(
  damaging_mutation, 
  paste0('temp/damaging_mutation.', release, '.all.csv'), sep = ',', quote = F)
# Save the ready to upload file to upload to taiga
write.table(
  other_mutation, 
  paste0('temp/other_mutation.', release, '.all.csv'), sep = ',', quote = F)
# Save the ready to upload file to upload to taiga
write.table(
  hotspot_mutation, 
  paste0('temp/hotspot_mutation.', release, '.all.csv'), sep = ',', quote = F)

# Validation

## Compare to previous release

I would run some checks here comparing the results to the previous releases MAF. Namely:

- Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
- Count the total number of mutations observed by position (group by chromosome, start position, end position and count the number of mutations)
- Look at specific differences between the two MAFs (join on DepMap_ID, Chromosome, Start position, End position, Variant_Type). I would do this for WES only

In [48]:
mutations = pd.read_csv('temp/mutations.'+release+'.all.csv')
damaging_mutation = pd.read_csv('temp/damaging_mutation.'+release+'.all.csv')
print(len(damaging_mutation))
other_mutation = pd.read_csv('temp/other_mutation.'+release+'.all.csv')
print(len(other_mutation))
hotspot_mutation = pd.read_csv('temp/hotspot_mutation.'+release+'.all.csv')
print(len(hotspot_mutation))

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
mutations.columns

In [ ]:
set(mutations.DepMap_ID) - set(mutations[~(mutations['CGA_WES_AC'].isna() & mutations['SangerWES_AC'].isna() & mutations['WGS_AC'].isna() & mutations['SangerRecalibWES_AC'].isna())].DepMap_ID)

In [ ]:
len(a)

In [ ]:
mutations[mutations.DepMap_ID=="ACH-000458"].sum(0)

In [ ]:
mutations[mutations["Hugo_Symbol"]=="ACOT4"][mutations['Start_position']==74058831]

In [ ]:
ac_data = mutations[[val for val in mutations.columns.values if '_AC' in val]]

In [ ]:
ac_names = ac_data.columns.values
ac_data = ac_data.values

In [ ]:
ac_data.shape[0]

## Do some checks and manual rescuing

In [ ]:
mutations[mutations.DepMap_ID=="ACH-003000"]

## check important mutations

In [ ]:
# check MOLM13, MV411 cell lines- The well known mutation status of FLT3

In [ ]:
# check TP53 mutation 

In [ ]:
toofew = 0
allnan = 0
for pos, val in enumerate(ac_data):
    i = 0
    print(str(100*pos/ac_data.shape[0]),end='\r')
    for p, v in enumerate(val):
        if v is np.nan:
            i+=1
    if i==7:
        mutations = mutations.drop[pos]
        allnan+=1

In [ ]:
allnan

### basic counts

In [ ]:
#Count the total number of mutations per cell line, split by type (SNP, INS, DEL)

In [ ]:
# Count the total number of mutations observed by position

Are mutation consistent?

In [ ]:
#  to check this, if you group all the mutations in the mutations table by Chromosome, Start_position, End_position, Reference_Allele, Tumor_Seq_Allele1 columns, they should all have the same annotation for the other columns (protein change, exac_af, etc...)

QC mutations, for a known dependency, check if it matches mutation of this gene. (if P53 is mutated, cannot have dependency on P53 or MDM2 MDM4/ inverse fir BRAF and KRAF to themselves)

In [ ]:
prevprevname,prevprevversion 

In [ ]:
mutations[mutations.DepMap_ID=="ACH-001546"][mutations.columns[-17:]]

In [31]:
prevprev= set(tc.get(name='depmap-mutation-calls-9be3', file= "depmap_"+prevprevname+"_mutation_calls", version = prevprevversion).DepMap_ID.tolist())

# uploading on taiga

In [25]:
gsheets = sheets.get(sheeturl).sheets[6].to_frame()
wes_dmc_embargo = [i for i in gsheets['WES_DMC_embargo'].values.tolist() if str(i) != "nan"]
wes_embargo = [i for i in gsheets['WES_embargo'].values.tolist() if str(i) != "nan"]
blacklist = [i for i in gsheets['blacklist'].values.tolist() if str(i) != "nan"]

In [ ]:
wes_embargo, wes_dmc_embargo, blacklist

In [ ]:
! cd .. && git clone https://github.com/broadinstitute/depmap-release-readmes.git && cd -

In [ ]:
! cd ../depmap-release-readmes && git pull

In [ ]:
!cd ../depmap-release-readmes/ && python3 make_new_release.py $release && git add . && git commit -m $release && git push 

In [ ]:
os.system('cd ../depmap-release-readmes && git pull && mv release-'+release+'/internal-'+release+'.txt ../ccle_processing/temp/README && cd -')

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutations-maf-35fe",
                 upload_file_path_dict={'temp/mutations.'+release+'.all.csv': 'TableCSV'}, 
                 dataset_description="""
# Mutations

filtered and unfiltered mutation files from Broad WES and Sanger WES data mapped to hg19
The MAF file for DepMap that includes all of the latest WES samples. This MAF is generated by merging CCLE (WGS, RNAseq, RD, HC) and Sanger (WES) data.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

version 1:  In 19Q1 the WES_AC column has been replaced by two columns, VA_WES_AC and CGA_WES_AC. We are currently using the Van Allen and CGA based pipeline to generate mutation calls. The CGA pipeline includes more filtering on the MAFs than VA and has a better INDEL caller. However, some of these filters may be removing some variants of interest that are still capture by the VA pipeline, which is why both a retained for now. DEPRECATED:  Missing the VA_WES_AC, CGA_WES_AC columns
version 2: 19Q1 data
version 3: 19Q2 data. We are no longer using the CCLE_WES_AC column. We are only using the CGA pipeline for mutation calls.
version 4: Updating to 19Q3interim DEPRECATED
version 5: Updating to 19Q3interim DEPRECATED
version 6: Updating to 19Q3interim
version 7: Updating to 19Q3 DEPRECATED
version 8: reparing the missing mutation problem DEPRECATED
version 9: reparing the missing column problem


version10:
Adding 52 new cell lines. 
Some cells lines have been flagged as:

version11:
adding missing cell lines

Adding 52 new cell lines. 
Some cells lines have been flagged as:

 - having bad looking copy ration plots = 
 - Genes having a similar CN value accross all []

version 12:

adding 8 new cell lines

version 13:

removing a wrong column

version 14:

adding 8 new cell lines. Adding .all. since we are soon going to release a restricted set of mutations. this one contains everything which is not necessarily what we want


genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
 """)

## Internal

In [ ]:
hotspot_mutation

In [ ]:
prevmut = tc.get(name='depmap-mutation-calls-9be3', version=24, file='depmap_'+prevname+'_mutation_calls.all')
print('shoud be None')
print(set(prevmut.DepMap_ID) - set(mutations.DepMap_ID))
print("new lines")
newlines = set(mutations.DepMap_ID) - set(prevmut.DepMap_ID) 
newlines

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutation-calls-9be3",
                 upload_file_path_dict={'temp/depmap_'+release+'_mutation_calls.all': 'TableCSV',
                                        'temp/damaging_mutation.all': 'NumericMatrixCSV',
                                        'temp/other_mutation.all': 'NumericMatrixCSV',
                                        'temp/hotspot_mutation.all': 'NumericMatrixCSV',
                                       },#'temp/README': 'Raw'},
                 dataset_description="""
# Internal Mutations

Mutation calls for Internal DepMap data

* Version 1 Internal 18Q1*

original source: `/xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_depMap_18Q1_maf_20180202.txt`
* Version 2-4 Internal 18Q2*

merged mutations and indels file (1,606 cell lines, including CCLE and Sanger WES reanalysis)
original source: 
`/xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_depMap_18q2_maf_20180502.txt`
Binary matrices:
- damaging: if isDeleterious is true
- missense: if isDeleterious is false
- hotspot: if missense and either TCGA or COSMIC hotspot
Version 2 contains the MAF file
* Version 5-6 Internal 18Q3*

version 5 deprecated

original source: `/xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_depMap_18q3_maf_20180716.txt`

Binary matrices:
- damaging: if isDeleterious is true
- missense: if isDeleterious is false
- hotspot: if missense and either TCGA or COSMIC hotspot
- Rows: cell line, Broad (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)

MAF file

* Version 7-8 Internal 18Q4*

version 8 just changes a column name in the MAF file from Broad_ID to DepMap_ID

original source: `/xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_DepMap_18Q4_maf_20181028.txt`

* Version 9-12 Internal 19Q1*

version 12 updates the column name from VA_WES_AC to CCLE_WES_AC

version 11+ uses an updated definition for hotspot mutations

version 12 contains the correct data for 19Q1

* Version 13 Internal 19Q2*

* Version 14-15 Internal 19Q3*

version 15 fixed entrez ids

* Version 16 Internal 19Q4*

adding 35 new cell lines.

* Version 16 Internal 19Q4*
uploading as matrices

* Version 17 Internal 19Q4*
removing unauthorized lines and setting as matrices

* Version 18 Internal 19Q4*
removing unauthorized lines and setting as matrices

* Version 19 Internal 20Q1*
uploading 8 new lines

* Version 20 Internal 20Q1*
removing unauthorized cl

* Version 21 Internal 20Q2*
uploading 8 new lines and adding .all to express the fact that this data is the aggregate of all different sequencing methods.

* Version 22 Internal 20Q2*
removing 2 cell lines

* Version 23 Internal 20Q3*
nothing different from 20Q2. no new cell lines

* Version 24 Internal 20Q2*
updating the blacklists

*** Variant annotation column ***

MAF file, added column (Variant_annotation) classifying each variant as either silent, damaging, other conserving, or other non-conserving, based on this mapping (old annotation from Variant_Classification column - new annotation):

Silent - silent
Splice_Site - damaging
Missense_Mutation - other non-conserving
Nonsense_Mutation - damaging
De_novo_Start_OutOfFrame - damaging
Nonstop_Mutation - other non-conserving
Frame_Shift_Del - damaging
Frame_Shift_Ins - damaging
In_Frame_Del - other non-conserving
In_Frame_Ins - other non-conserving
Stop_Codon_Del - other non-conserving
Stop_Codon_Ins - other non-conserving
Start_Codon_SNP - damaging
Start_Codon_Del - damaging
Start_Codon_Ins - damaging
5'Flank - other conserving
Intron - other conserving
IGR - other conserving
3'UTR - other conserving
5'UTR - other conserving
Binary matrices:

- damaging: if damaging
- other: if other conserving or other non-conserving
- hotspot: if it is not a silent mutation and is either TCGA or COSMIC hotspot
- Rows: cell line, DepMap (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)

NEW LINES:
"""+newlines)

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_internal, 'depmap-mutation-calls-9be3', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls'),])#('README','README')])
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', 'depmap-mutation-calls-9be3', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls')])

## DMC

In [ ]:
os.system('cd ../depmap-release-readmes && git pull && mv release-'+releAse+'/dmc-'+releAse+'.txt ../ccle_processing/temp/README && cd -')

In [9]:
print(len(mutations))
mutations = mutations[~mutations.DepMap_ID.isin(wes_embargo)]
print(len(mutations))
mutations.to_csv('temp/depmap_'+release+'_mutation_calls.all', index=False)
print(len(damaging_mutation))
damaging_mutation = damaging_mutation[~damaging_mutation.index.isin(wes_embargo)]
print(len(damaging_mutation))
damaging_mutation.to_csv('temp/damaging_mutation.all')
print(len(other_mutation))
other_mutation = other_mutation[~other_mutation.index.isin(wes_embargo)]
print(len(other_mutation))
other_mutation.to_csv('temp/other_mutation.all',)
print(len(hotspot_mutation))
hotspot_mutation = hotspot_mutation[~hotspot_mutation.index.isin(wes_embargo)]
print(len(hotspot_mutation))
hotspot_mutation.to_csv('temp/hotspot_mutation.all',)

1301656
1297418
1757
1744
1758
1745
1725
1712


In [12]:
prevmut = tc.get(name='depmap-mutation-calls-dfce', version=15, file='depmap_'+prevname+'_mutation_calls')
print('shoud be None')
print(set(prevmut.DepMap_ID) - set(mutations.DepMap_ID))
print("new lines")
newlines = set(mutations.DepMap_ID) - set(prevmut.DepMap_ID) 
newlines

[##################]100% |  40.4 MiB/s | 277.3 MiB / 277.3 MiB | Time:  0:00:06


shoud be None
set()
new lines


{'ACH-001533', 'ACH-001574', 'ACH-002021', 'ACH-002065'}

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutation-calls-dfce",
                 upload_file_path_dict={'temp/depmap_'+release+'_mutation_calls.all': 'TableCSV',
                                        'temp/damaging_mutation.all': 'NumericMatrixCSV',
                                        'temp/other_mutation.all': 'NumericMatrixCSV',
                                        'temp/hotspot_mutation.all': 'NumericMatrixCSV',
                                       },#'temp/README': 'Raw'},
                 dataset_description="""
# DMC Mutations

* Version 1-5 DMC 19Q1*

version 5 is a one-off portal thing because dmc wanted to be able to plot if a gene has any mutation as one-hot encoded value in the x/y axes of the data explorer It adds the any_mutation matrix, but does not change the others. Code used to generate:

```
from taigapy import TaigaClient

c = TaigaClient()

dmc_19q1_mutation_taiga_root = "depmap-mutation-calls-dfce.3/"
other_matrix = c.get(dmc_19q1_mutation_taiga_root + "other_mutation")
damaging_matrix = c.get(dmc_19q1_mutation_taiga_root + "damaging_mutation")
hotspot_matrix = c.get(dmc_19q1_mutation_taiga_root + "hotspot_mutation")

df = other_matrix.append(damaging_matrix)
df = df.groupby(level=0).sum()

df = df.append(hotspot_matrix)
df = df.groupby(level=0).sum()

df[df > 1] = 1

df.to_csv('any_mutation.csv')
```
The code uses version 3 because the dmc portal was using version 3

version 4 updates the column name from VA_WES_AC to CCLE_WES_AC

version 3 has an updated definition for hotspot mutations

version 2+ contains the correct data for 19Q1

* Version 6 DMC 19Q2*

* Version 7-8 DMC 19Q3*
version 8 fixed entrez ids

* Version 9 DMC 19Q4*
adding 52 new cell lines.

* Version 10 DMC 19Q4*
removing unauthorized lines and setting as matrices

* Version 11 DMC 19Q4*
removing unauthorized lines and setting as matrices

* Version 12 Internal 20Q1*
uploading 8 new lines

* Version 13 Internal 20Q1*
removing unauthorized cl

* Version 14 Internal 20Q2*
uploading 8 new lines and adding .all to express the fact that this data is the aggregate of all different sequencing methods.

* Version 15 Internal 20Q2*
removing 2 lines

* Version 15 Internal 20Q3*
nothing different from 20Q2. no new cell lines

* Version 15 Internal 20Q3*
updating the blacklists


MAF file, added column (Variant_annotation) classifying each variant as either silent, damaging, other conserving, or other non-conserving, based on this mapping (old annotation from Variant_Classification column - new annotation):

Silent - silent
Splice_Site - damaging
Missense_Mutation - other non-conserving
Nonsense_Mutation - damaging
De_novo_Start_OutOfFrame - damaging
Nonstop_Mutation - other non-conserving
Frame_Shift_Del - damaging
Frame_Shift_Ins - damaging
In_Frame_Del - other non-conserving
In_Frame_Ins - other non-conserving
Stop_Codon_Del - other non-conserving
Stop_Codon_Ins - other non-conserving
Start_Codon_SNP - damaging
Start_Codon_Del - damaging
Start_Codon_Ins - damaging
5'Flank - other conserving
Intron - other conserving
IGR - other conserving
3'UTR - other conserving
5'UTR - other conserving
Binary matrices:
- damaging: if damaging
- other: if other conserving or other non-conserving
- hotspot: if it is not a silent mutation and is either TCGA or COSMIC hotspot
- Rows: cell line, DepMap (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)

NEW LINES:
"""+newlines)

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_dmc, 'depmap-mutation-calls-dfce', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls'),])#('README','README')])

## Public

In [ ]:
os.system('cd ../depmap-release-readmes && git pull && mv release-'+releAse+'/public-'+releAse+'.txt README && cd -')

In [27]:

#damaging_mutation
mutations=depmap_20Q3_mutation_calls
#hotspot_mutation
#other_mutation

In [35]:
print(len(mutations))
mutations = mutations[mutations.DepMap_ID.isin(prevprev)]
mutations = mutations[~mutations.DepMap_ID.isin(wes_dmc_embargo)]
print(len(mutations))
mutations.to_csv('temp/depmap_'+release+'_mutation_calls.all', index=False)
print(len(damaging_mutation))
damaging_mutation = damaging_mutation[damaging_mutation.index.isin(prevprev)]
damaging_mutation = damaging_mutation[~damaging_mutation.index.isin(wes_dmc_embargo)]
print(len(damaging_mutation))
damaging_mutation.to_csv('temp/damaging_mutation.all')
print(len(other_mutation))
other_mutation = other_mutation[other_mutation.index.isin(prevprev)]
other_mutation = other_mutation[~other_mutation.index.isin(wes_dmc_embargo)]
print(len(other_mutation))
other_mutation.to_csv('temp/other_mutation.all')
print(len(hotspot_mutation))
hotspot_mutation = hotspot_mutation[hotspot_mutation.index.isin(prevprev)]
hotspot_mutation = hotspot_mutation[~hotspot_mutation.index.isin(wes_dmc_embargo)]
print(len(hotspot_mutation))
hotspot_mutation.to_csv('temp/hotspot_mutation.all')

1297418
1297418
1744
1744
1745
1745
1712
1712


In [36]:
prevmut = tc.get(name='depmap-mutation-calls-9a1a', version=18, file='depmap_'+prevname+'_mutation_calls')
print('shoud be None')
ermgency_removed = set(prevmut.DepMap_ID) - set(mutations.DepMap_ID)
print(ermgency_removed) 
print("new lines")
newlines = set(mutations.DepMap_ID) - set(prevmut.DepMap_ID) 
newlines

shoud be None
set()
new lines


{'ACH-001533', 'ACH-001574', 'ACH-002021', 'ACH-002065'}

In [38]:
description="""
# Public Mutations

Mutation calls for Public DepMap data

* Version 1 Public 18Q1*

original source: CCLE data portal
* Version 2 Public 18Q2*

merged mutations and indels file (1,549 cell lines total, including data for 63 newly released cell lines)
original source: `/xchip/ccle_dist/public/DepMap_18Q2/CCLE_DepMap_18Q2_maf_20180502.txt`
* Version 3-4 Public 18Q3*

version 3 deprecated

original source: `/xchip/ccle_dist/public/DepMap_18Q3/CCLE_DepMap_18q3_maf_20180718.txt`

Binary matrices:
damaging: if isDeleterious is true
missense: if isDeleterious is false
hotspot: if missense and either TCGA or COSMIC hotspot
Rows: cell line, Broad (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)

MAF file

* Version 5 Public 18Q4*

original source: `/xchip/ccle_dist/public/DepMap_18Q4/CCLE_DepMap_18q4_maf_20181029.txt`

* Version 6-9 Public 19Q1*

version 9 updates the column name from VA_WES_AC to CCLE_WES_AC

version 8 uses an updated definition for hotspot mutations

version 9 contains the correct data for 19Q1

* Version 10 Public 19Q2*

* Version 11-12 Public 19Q3*

version 12 fixed entrez ids

* Version 13 Public 19Q4*

adding 52 new cell lines

* Version 14 Public 19Q4*
removing unauthorized lines and setting matrices

* Version 15 Public 20Q1*
adding 8 new lines 

* Version 16 Public 20Q1*
removing an unauthorized line

* Version 17 Internal 20Q2*
uploading 8 new lines and adding .all to express the fact that this data is the aggregate of all different sequencing methods.

* Version 18 Internal 20Q2*
removing 2 lines

* Version 19 Internal 20Q3*
nothing different from 20Q2. no new cell lines

* Version 20 Internal 20Q3*
updating the blacklists

* Version 21 Internal 20Q3*
updating the dmc

* Version 22 Internal 20Q3*
readding two already released samples to the public list

MAF file, added column (Variant_annotation) classifying each variant as either silent, damaging, other conserving, or other non-conserving, based on this mapping (old annotation from Variant_Classification column - new annotation):

Silent - silent
Splice_Site - damaging
Missense_Mutation - other non-conserving
Nonsense_Mutation - damaging
De_novo_Start_OutOfFrame - damaging
Nonstop_Mutation - other non-conserving
Frame_Shift_Del - damaging
Frame_Shift_Ins - damaging
In_Frame_Del - other non-conserving
In_Frame_Ins - other non-conserving
Stop_Codon_Del - other non-conserving
Stop_Codon_Ins - other non-conserving
Start_Codon_SNP - damaging
Start_Codon_Del - damaging
Start_Codon_Ins - damaging
5'Flank - other conserving
Intron - other conserving
IGR - other conserving
3'UTR - other conserving
5'UTR - other conserving
Binary matrices:

- damaging: if damaging
- other: if other conserving or other non-conserving
- hotspot: if it is not a silent mutation and is either TCGA or COSMIC hotspot
- Rows: cell line, DepMap (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)

NEW LINES:
"""+str(newlines)

if len(ermgency_removed):
    description+="""
    
    !! WE REMOVED!!:
    """+str(ermgency_removed)

tc.update_dataset(dataset_permaname="depmap-mutation-calls-9a1a",
                 upload_file_path_dict={'temp/depmap_'+release+'_mutation_calls.all': 'TableCSV',
                                        'temp/damaging_mutation.all': 'NumericMatrixCSV',
                                        'temp/other_mutation.all': 'NumericMatrixCSV',
                                        'temp/hotspot_mutation.all': 'NumericMatrixCSV',
                                       },#'temp/README': 'Raw'},
                 dataset_description=description)

Uploading depmap_20Q3_mutation_calls...
hitting https://cds.team/taiga/api/datafile/00b151ca35a14f6d9f1be95ef24ea368
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: depmap_20Q3_mutation

'8cc4900a50874d8593b0bfc591001360'

In [39]:
# To add to a virtual dataset
AddToVirtual(virtual_public, 'depmap-mutation-calls-9a1a', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls'),])#('README','README')])

[('CCLE_mutations', 'depmap-mutation-calls-9a1a.22/depmap_20Q3_mutation_calls'), ('CCLE_gene_cn', 'depmap-wes-cn-data-97cc.34/public_20Q3_gene_cn'), ('Achilles_gene_effect_unscaled', 'avana-public-tentative-20q3-3e73.5/gene_effect_unscaled'), ('Achilles_high_variance_genes', 'avana-public-tentative-20q3-3e73.5/high_variance_genes'), ('Achilles_guide_efficacy', 'avana-public-tentative-20q3-3e73.5/guide_efficacy'), ('CCLE_fusions_unfiltered', 'gene-fusions-6212.14/unfiltered_fusions_20Q3'), ('common_essentials', 'avana-public-tentative-20q3-3e73.5/essential_genes'), ('Achilles_logfold_change_failures', 'avana-public-tentative-20q3-3e73.5/logfold_change_failures'), ('CCLE_expression', 'depmap-rnaseq-expression-data-ccd0.25/public_20Q3_proteincoding_tpm'), ('Achilles_raw_readcounts', 'avana-public-tentative-20q3-3e73.5/raw_readcounts'), ('Achilles_raw_readcounts_failures', 'avana-public-tentative-20q3-3e73.5/raw_readcounts_failing'), ('README', 'public-20q3-3d35.22/README'), ('CCLE_segment